# Welcome 

In [1]:
import importlib
importlib.invalidate_caches()

In [2]:
import os
os.environ["OPENAI_API_BASE"]= "https://api.openai.com/v1"

In [6]:
# Install and import required libraries
import nest_asyncio
import asyncio
nest_asyncio.apply()  # Required for async in notebooks

from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.genai import types

# Constants — define application, user, and session identifiers
APP_NAME = "adk-welcome"    # Name of the ADK application
USER_ID = "wew"           # Identifier for the current user
SESSION_ID = "welcome_session" # Identifier for the conversation session

In [12]:
from google.adk.agents import LlmAgent
from google.adk.models.lite_llm import LiteLlm

AGENT_MODEL = LiteLlm(model="openai/gpt-4o-mini")

welcome_agent = LlmAgent(
    name="WelcomeAgent",
    description=f"An agent that welcomes the user named {USER_ID}",
    instruction="Always greet the user with salam politely and offer assistance.",
    model=AGENT_MODEL
)

print(f"Agent '{welcome_agent.name}' created.")


# set up session service and create session 
# Set up session service and create a session
session_service = InMemorySessionService()
await session_service.create_session(
    app_name=APP_NAME, 
    user_id=USER_ID, 
    session_id=SESSION_ID
)

Agent 'WelcomeAgent' created.


Session(id='welcome_session', app_name='adk-welcome', user_id='wew', state={}, events=[], last_update_time=1766673500.435541)

In [13]:
# Set up a runner to orchestrate the agent
runner = Runner(agent=welcome_agent, app_name=APP_NAME, session_service=session_service)

In [14]:
# Define an asynchronous function to send a query to the agent and handle its response
async def call_agent_async(query: str):
    print(f"\n>>> User Query: {query}")

    # Package the user's query into ADK format
    content = types.Content(role='user', parts=[types.Part(text=query)])
    final_response_text = "Agent did not produce a final response."

    # Iterate through streamed agent responses
    async for event in runner.run_async(user_id=USER_ID, session_id=SESSION_ID, new_message=content):
        if event.is_final_response(): # Check if this is the final message from the agent
            if event.content and event.content.parts:
                final_response_text = event.content.parts[0].text # Extract response text
            break # Stop listening after final response is received

    print(f"<<< Agent Response: {final_response_text}")

# Run the interaction
await call_agent_async("Hi there!")


>>> User Query: Hi there!
<<< Agent Response: Wa alaikum assalam! How can I assist you today?


In [16]:
await call_agent_async("Assalamu alaikum?")


>>> User Query: Assalamu alaikum?
<<< Agent Response: Wa alaikum assalam! How can I assist you today?
